<a href="https://colab.research.google.com/github/Mihir-Panjikar/Legal_Document_Simplication_System/blob/dev/Notebook/Llama_legal_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers
!pip install --upgrade bitsandbytes
!pip install --upgrade huggingface_hub

In [ ]:
from google.colab import userdata
read_token = userdata.get('read_hugging_face')

In [ ]:
!huggingface-cli login --token $read_token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `Read_access` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Read_access`


In [ ]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True  # Enable CPU offload
)

quantized_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=quantization_config
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

def format_messages(messages):
    formatted = ""
    for msg in messages:
        formatted += f"{msg['role'].capitalize()}: {msg['content']}\n\n"
    return formatted

input_text = format_messages([
    {"role": "system", "content": """You are an expert in summarization and legal document simplification. Your task is to summarize the following agreement in a way that is easy for a layperson to understand. The summary should include all key details while using simple, clear language and relevant real-life examples where needed. Ensure that no critical information is lost.

Key requirements for the summary:

Use plain English, avoiding complex legal terms.
Provide relevant examples to make the agreement more understandable.
Maintain all important details, such as duration, termination clauses, liability, dispute resolution, and governing law.
Ensure the summary remains concise but does not miss crucial points."""},
    {"role": "user", "content": """This Agreement shall commence on the Effective Date and shall continue for a period of five (5) years unless earlier terminated in accordance with the provisions set forth herein. Either Party may terminate this Agreement upon thirty (30) days prior written notice to the other Party in the event of a material breach that remains uncured for a period of fifteen (15) days following written notice of such breach. In the event of termination, both Parties shall fulfill any obligations incurred prior to the effective date of termination. Furthermore, neither Party shall be liable to the other for any indirect, consequential, or punitive damages arising out of or in connection with this Agreement, except in cases of willful misconduct or gross negligence. Any disputes arising under this Agreement shall be resolved through binding arbitration in accordance with the rules of the American Arbitration Association, and the prevailing Party shall be entitled to recover reasonable attorneys’ fees and costs incurred in connection with such arbitration. This Agreement shall be governed by and construed in accordance with the laws of the State of New York, without regard to its conflict of law principles. If any provision of this Agreement is found to be invalid or unenforceable, the remaining provisions shall continue in full force and effect. No modification, amendment, or waiver of any provision of this Agreement shall be effective unless in writing and signed by both Parties. This Agreement constitutes the entire agreement between the Parties with respect to the subject matter herein and supersedes all prior and contemporaneous agreements, representations, and understandings, whether written or oral."""}
])

input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

output = quantized_model.generate(**input_ids, max_new_tokens=256)

print(tokenizer.decode(output[0], skip_special_tokens=True))
